In [232]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

import cufflinks as cf
import numpy as np
import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

### Read data and transform datatypes to datetime and adding additional time columns

In [233]:
# read in data, expand df to show all columns
df_main = pd.read_csv('C:\\Users\\wramsey\\Desktop\\tiffany.csv')
pd.set_option('max_columns', None)

# Change datatypes of all time-based columns
time_columns = ['sys_updated_on', 'sys_created_on', 'closed_at', 'opened_at', 'business_duration', 'reopened_time',
                'resolved_at', 'work_start', 'calendar_duration']

for item in time_columns:
    df_main[item] = pd.to_datetime(df_main[item])

# Create additional time columns
df_main['opened_at'] = df_main['opened_at'] - timedelta(hours=5)
df_main['resolved_at'] = df_main['resolved_at'] - timedelta(hours=5)
df_main['opened_at_hour'] = df_main['opened_at'].dt.hour
df_main['opened_at_month'] = df_main['opened_at'].dt.month
df_main['opened_at_day'] = df_main['opened_at'].dt.day
df_main['opened_at_day_of_year'] = df_main['opened_at'].dt.dayofyear
df_main['opened_at_clone'] = df_main['opened_at']
df_main['resolution_hour'] = df_main['resolved_at'].dt.hour
df_main['resolution_time_length'] = df_main.resolved_at - df_main.opened_at
df_main['resolution_time_total_hours'] = df_main['resolution_time_length'].dt.total_seconds()/3600
# print(type(df_main['opened_at_clone'].iloc[0]))
df_main[['opened_at', 'task_effective_number', 'opened_at_hour', 'opened_at_month', 'opened_at_day', 'opened_at_day_of_year',
         'opened_at_clone', 'resolution_time_length',
         'resolution_time_total_hours']].head(5).sort_values('opened_at_day').head(2)

# place all transformations here
df_main = df_main.replace(['2d5d343adb2d17009f9aec51ca9619e9'],['five9'])
df_main = df_main.replace(['55da5d83db3057007f3dec51ca96190c'],['Unknown_CI'])

# Get today's date
sixty_days = datetime.today() - timedelta(60)

# to only include results after today's data
df_main = df_main[ df_main['opened_at'] > sixty_days.date().strftime('%Y-%m-%d') ]

### What is the timeframe of this data

In [234]:
# to pull days as int (for calculations)
# print((df_main['opened_at'].max() - df_main['opened_at'].min()).days)
print('Total number of incidents tracked within this selection: ' + str(df_main['opened_at'].count()))
print('earliest incident:        '+str(df_main['opened_at'].dt.date.min()))
print('latest incident:          '+str(df_main['opened_at'].dt.date.max()))
print('timespan/range of earliest to latest:   '+str(df_main['opened_at'].max() - df_main['opened_at'].min()))

Total number of incidents tracked within this selection: 1333
earliest incident:        2021-02-25
latest incident:          2021-04-26
timespan/range of earliest to latest:   60 days 07:59:49


### How many INC are opened per hour - Total Day Overview

In [235]:
df_inc_per_hr = df_main['task_effective_number'].groupby(df_main['opened_at'].dt.hour).count()
# df_inc_per_hr

fig = px.bar(df_inc_per_hr, x=df_inc_per_hr.index, y='task_effective_number',
             labels={'opened_at':'Hour Opened - 24hr Clock', 'task_effective_number':'Total Count of Incidents'},
            title='INC Opened Per Hour (Past 60 Days) - Vertical Line at 6pm')
fig.add_vline(x=16.5, line_dash="dash", line_color="green")
fig.show()

### Total count of INC opened per day - After 6pm

In [236]:
# Set to after 6pm only
df_main = df_main[ df_main['opened_at_hour'] > 17]

df_daily = df_main.copy(deep=True)
df_daily.set_index('opened_at_clone', inplace=True)
df_daily_resample_b = df_daily['task_effective_number'].resample('B').count()

fig = px.line(df_daily_resample_b, x=df_daily_resample_b.index, y=df_daily['task_effective_number'].resample('B').count(),
              title='Count of Total INC Per Day After 6pm (Weekends not included)')
fig.show()

### Who has opened the most tickets overall - after 6pm? (by count)

In [237]:
# show number of tickets by INC-caller 
fig = px.bar(df_main['sys_created_by'].value_counts(), x=df_main['sys_created_by'].value_counts().index,
             y='sys_created_by', labels={'x':'Username', 'sys_created_by':'Number of INC opened'})
fig.update_xaxes(range=[0,50])
fig.show()

### Distribution of INC type (as percentage)

In [238]:
# df_lots_of_time['subcategory'].value_counts().head()
fig = px.bar(df_main['subcategory'][df_main['subcategory']!=0].value_counts(normalize=True, ascending=False),
             x=df_main['subcategory'][df_main['subcategory']!=0].value_counts().index,
             y='subcategory')
fig.show()